# Azure AI-agenter med Model Context Protocol (MCP)-understøttelse - Python

Denne notebook demonstrerer, hvordan man bruger Azure AI-agenter med Model Context Protocol (MCP)-værktøjer i Python. Den viser, hvordan man opretter en intelligent agent, der kan udnytte eksterne MCP-servere (som Microsoft Learn) for forbedrede funktioner ved hjælp af nøglefri autentifikation.


## Installer Nødvendige Python Pakker

Først skal vi installere de nødvendige Python-pakker:
- **azure-ai-projects**: Core Azure AI Projects SDK
- **azure-ai-agents**: Azure AI Agents SDK til oprettelse og administration af agenter
- **azure-identity**: Giver nøglefri autentifikation ved hjælp af DefaultAzureCredential
- **mcp**: Implementering af Model Context Protocol for Python


## Fordele ved nøglefri godkendelse

Denne notebook demonstrerer **nøglefri godkendelse**, som giver flere fordele:
- ✅ **Ingen API-nøgler at administrere** - Bruger Azure-identitetsbaseret godkendelse
- ✅ **Forbedret sikkerhed** - Ingen hemmeligheder gemt i kode eller konfigurationsfiler
- ✅ **Automatisk rotation af legitimationsoplysninger** - Azure håndterer livscyklusstyring af legitimationsoplysninger
- ✅ **Rollebaseret adgangskontrol** - Bruger Azure RBAC til detaljerede tilladelser
- ✅ **Understøttelse af flere miljøer** - Fungerer problemfrit på tværs af udvikling og produktion

`DefaultAzureCredential` vælger automatisk den bedst tilgængelige kilde til legitimationsoplysninger:
1. **Managed Identity** (når det kører i Azure)
2. **Azure CLI** legitimationsoplysninger (under lokal udvikling)
3. **Visual Studio** legitimationsoplysninger
4. **Miljøvariabler** (hvis konfigureret)
5. **Interaktiv browser**-godkendelse (som en sidste udvej)


## Opsætning af nøglefri autentifikation

**Forudsætninger for nøglefri autentifikation:**

### Til lokal udvikling:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Til Azure-miljøer:
- Aktiver **System-tildelt Managed Identity** på din Azure-ressource
- Tildel passende **RBAC-roller** til den administrerede identitet:
  - `Cognitive Services OpenAI User` for adgang til Azure OpenAI
  - `AI Developer` for adgang til Azure AI-projekter

### Miljøvariabler (valgfrit):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Ingen API-nøgler eller forbindelsesstrenge nødvendige!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importer nødvendige biblioteker

Importer de nødvendige Python-moduler:
- **os, time**: Standard Python-biblioteker til miljøvariabler og forsinkelser
- **AIProjectClient**: Hovedklienten til Azure AI Projects
- **DefaultAzureCredential**: Nøglefri autentificering til Azure-tjenester
- **MCP-relaterede klasser**: Til oprettelse og administration af MCP-værktøjer og håndtering af godkendelser


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurer MCP-serverindstillinger

Opsæt MCP-serverkonfigurationen ved hjælp af miljøvariabler med standardværdier som fallback:
- **MCP_SERVER_URL**: URL'en til MCP-serveren (standard er Microsoft Learn API)
- **MCP_SERVER_LABEL**: En etiket til at identificere MCP-serveren (standard er "mslearn")

Denne tilgang muliggør fleksibel konfiguration på tværs af forskellige miljøer.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Opret Azure AI Project Client (Nøglefri Godkendelse)

Initialiser Azure AI Project-klienten ved hjælp af **nøglefri godkendelse**:
- **endpoint**: URL'en til Azure AI Foundry-projektets endpoint
- **credential**: Bruger `DefaultAzureCredential()` til sikker, nøglefri godkendelse
- **Ingen API-nøgler påkrævet**: Finder og bruger automatisk den bedst tilgængelige godkendelse

**Godkendelsesflow:**
1. Tjekker for Managed Identity (i Azure-miljøer)
2. Fald tilbage til Azure CLI-legitimationsoplysninger (til lokal udvikling)
3. Bruger andre tilgængelige godkendelseskilder efter behov

Denne tilgang eliminerer behovet for at administrere API-nøgler eller forbindelsesstrenge i din kode.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Opret MCP-værktøjsdefinition

Opret et MCP-værktøj, der forbinder til Microsoft Learn MCP-serveren:
- **server_label**: Identifikator for MCP-serveren
- **server_url**: URL-endepunkt for MCP-serveren
- **allowed_tools**: Valgfri liste til at begrænse, hvilke værktøjer der kan bruges (tom liste tillader alle værktøjer)

Dette værktøj giver agenten mulighed for at få adgang til Microsoft Learn-dokumentation og ressourcer.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Opret agent og udfør samtale (Nøglefri arbejdsgang)

Denne omfattende sektion demonstrerer den komplette **nøglefri agentarbejdsgang**:

1. **Opret AI-agent**: Opsæt en agent med GPT-4.1 nano-model og MCP-værktøjer
2. **Opret tråd**: Etabler en samtaletråd til kommunikation
3. **Send besked**: Spørg agenten om forskellene mellem Azure OpenAI og OpenAI
4. **Håndter værktøjsgodkendelser**: Godkend automatisk MCP-værktøjsanmodninger, når det er nødvendigt
5. **Overvåg udførelse**: Følg agentens fremskridt og håndter eventuelle nødvendige handlinger
6. **Vis resultater**: Vis samtalen og detaljer om værktøjsbrug

**Nøglefri funktioner:**
- ✅ **Ingen hardkodede hemmeligheder** - Al autentificering håndteres af Azure-identitet
- ✅ **Sikker som standard** - Bruger rollebaseret adgangskontrol
- ✅ **Forenklet implementering** - Ingen behov for håndtering af legitimationsoplysninger
- ✅ **Revisionsvenlig** - Al adgang spores via Azure-identitet

Agenten vil bruge MCP-værktøjer til at få adgang til Microsoft Learn-ressourcer med fuld sikkerhed og uden behov for API-nøgleadministration.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
